In [1]:
import sklearn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import time
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
from scipy import interpolate
from ulity import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
np.random.seed(111)

conv_1_shape = '3*3*1*32'
pool_1_shape = 'None'

conv_2_shape = '3*3*1*64'
pool_2_shape = 'None'

conv_3_shape = '3*3*1*128'
pool_3_shape = 'None'

conv_4_shape = 'None'
pool_4_shape = 'None'


window_size = 200*5
n_lstm_layers = 2
# full connected parameter
fc_size = 1024
n_fc_in = 1024
n_fc_out = 1024

dropout_prob = 0.5

calibration = 'N'
norm_type='2D'
regularization_method = 'dropout'
enable_penalty = False

output_dir 	= "roc1"
output_file = "roc2"

dataset_dir = "./result/"

with open(dataset_dir+"D3_data3_neg2.pkl", "rb") as fp:
  	datasets = pickle.load(fp)
with open(dataset_dir+"D3_label3_neg2.pkl", "rb") as fp:
  	labels = pickle.load(fp)

datasets = datasets.reshape(len(datasets), window_size, 10, 11, 1)
print('labels', len(labels))
one_hot_labels = np.array(list(pd.get_dummies(labels)))
print(one_hot_labels)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(datasets, labels, test_size=0.1, stratify = labels)


labels 3330
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]


In [3]:
train_x = X_train
train_y = y_train

train_sample = len(train_x)
print("train sample:", train_sample)

# test_x = datasets[~split]
# test_y = labels[~split]

test_x = X_test
test_y = y_test

test_sample = len(test_x)
print("test sample:", test_sample)

print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")

print(
    "**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")

# input parameter
input_channel_num = 1

input_height = 10
input_width = 11

n_labels = 15

# training parameter
lambda_loss_amount = 0.0001
training_epochs = 100
batch_size = 30
batch_num_per_epoch = train_x.shape[0] // batch_size

accuracy_batch_size = 30
train_accuracy_batch_num = train_x.shape[0] // accuracy_batch_size
test_accuracy_batch_num = test_x.shape[0] // accuracy_batch_size

# kernel parameter
kernel_height_1st = 3
kernel_width_1st = 3

kernel_height_2nd = 3
kernel_width_2nd = 3

kernel_height_3rd = 3
kernel_width_3rd = 3

kernel_stride = 1
conv_channel_num = 32
# pooling parameter
pooling_height = 2
pooling_width = 2

pooling_stride = 2

# algorithn parameter
global_step = tf.Variable(0, trainable=False)
initial_learning_rate = 0.001  # 初始学习率  # 初始学习率太大了，容易造成结果波动，但是不知道会不会鲁棒性好？

learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                           global_step=global_step,
                                           decay_steps=10, decay_rate=0.9)


# learning_rate = 1e-4

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W, kernel_stride):
    # API: must strides[0]=strides[4]=1
    return tf.nn.conv2d(x, W, strides=[1, kernel_stride, kernel_stride, 1], padding='SAME')


def apply_conv2d(x, filter_height, filter_width, in_channels, out_channels, kernel_stride):
    weight = weight_variable([filter_height, filter_width, in_channels, out_channels])
    bias = bias_variable([out_channels])  # each feature map shares the same weight and bias
    return tf.nn.elu(tf.add(conv2d(x, weight, kernel_stride), bias))


def apply_max_pooling(x, pooling_height, pooling_width, pooling_stride):
    # API: must ksize[0]=ksize[4]=1, strides[0]=strides[4]=1
    return tf.nn.max_pool(x, ksize=[1, pooling_height, pooling_width, 1],
                          strides=[1, pooling_stride, pooling_stride, 1], padding='SAME')


def apply_fully_connect(x, x_size, fc_size):
    fc_weight = weight_variable([x_size, fc_size])
    fc_bias = bias_variable([fc_size])
    return tf.nn.elu(tf.add(tf.matmul(x, fc_weight), fc_bias))


def apply_readout(x, x_size, readout_size):
    readout_weight = weight_variable([x_size, readout_size])
    readout_bias = bias_variable([readout_size])
    return tf.add(tf.matmul(x, readout_weight), readout_bias)


print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions End **********\n")

print("**********(" + time.asctime(time.localtime(time.time())) + ") Define NN structure Begin: **********\n")

# input placeholder
X = tf.placeholder(tf.float32, shape=[None, input_height, input_width, input_channel_num], name='X')
Y = tf.placeholder(tf.float32, shape=[None, n_labels], name='Y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
phase_train = tf.placeholder(tf.bool, name='phase_train')

# first CNN layer
conv_1 = apply_conv2d(X, kernel_height_1st, kernel_width_1st, input_channel_num, conv_channel_num, kernel_stride)
# pool_1 = apply_max_pooling(conv_1, pooling_height, pooling_width, pooling_stride)
print(conv_1.shape)
# second CNN layer
conv_2 = apply_conv2d(conv_1, kernel_height_2nd, kernel_width_2nd, conv_channel_num, conv_channel_num * 2,
                      kernel_stride)
# pool_2 = apply_max_pooling(conv_2, pooling_height, pooling_width, pooling_stride)
print(conv_2.shape)
# third CNN layer
conv_3 = apply_conv2d(conv_2, kernel_height_3rd, kernel_width_3rd, conv_channel_num * 2, conv_channel_num * 4,
                      kernel_stride)
# fully connected layer
print(conv_3.shape)
shape = conv_3.get_shape().as_list()

pool_2_flat = tf.reshape(conv_3, [-1, shape[1] * shape[2] * shape[3]])
fc = apply_fully_connect(pool_2_flat, shape[1] * shape[2] * shape[3], fc_size)

# dropout regularizer
# Dropout (to reduce overfitting; useful when training very large neural network)
# We will turn on dropout during training & turn off during testing

fc_drop = tf.nn.dropout(fc, keep_prob)

# fc_drop size [batch_size*window_size, fc_size]
# lstm_in size [batch_size, window_size, fc_size]
lstm_in = tf.reshape(fc_drop, [-1, window_size, fc_size])

cells = []
for _ in range(n_lstm_layers):
    cell = tf.contrib.rnn.BasicLSTMCell(n_fc_in, forget_bias=1.0, state_is_tuple=True)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    cells.append(cell)
lstm_cell = tf.contrib.rnn.MultiRNNCell(cells)

init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)


output, states = tf.nn.dynamic_rnn(lstm_cell, lstm_in, initial_state=init_state, time_major=False)

output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')
rnn_output = output[-1]

shape_rnn_out = rnn_output.get_shape().as_list()
# fc_out ==> [batch_size, n_fc_out]
fc_out = apply_fully_connect(rnn_output, shape_rnn_out[1], n_fc_out)

# keep_prob = tf.placeholder(tf.float32)
fc_drop = tf.nn.dropout(fc_out, keep_prob)

# readout layer
y_ = apply_readout(fc_drop, shape_rnn_out[1], n_labels)
y_pred = tf.argmax(tf.nn.softmax(y_), 1, name="y_pred")
y_posi = tf.nn.softmax(y_, name="y_posi")

# l2 regularization
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
)

if enable_penalty:
    # cross entropy cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y) + l2, name='loss')
else:
    # cross entropy cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y), name='loss')

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
add_global = global_step.assign_add(1)

# get correctly predicted object and accuracy
correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y_), 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
saver = tf.train.Saver()

print("**********(" + time.asctime(time.localtime(time.time())) + ") Define NN structure End **********\n")

print("**********(" + time.asctime(time.localtime(time.time())) + ") Train and Test NN Begin: **********\n")
# run
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as session:
    session.run(tf.global_variables_initializer())
    train_accuracy_save = np.zeros(shape=[0], dtype=float)
    test_accuracy_save = np.zeros(shape=[0], dtype=float)
    test_loss_save = np.zeros(shape=[0], dtype=float)
    train_loss_save = np.zeros(shape=[0], dtype=float)
    for epoch in range(training_epochs):
        cost_history = np.zeros(shape=[0], dtype=float)
        for b in range(batch_num_per_epoch):
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :, :]
            batch_x = batch_x.reshape(len(batch_x) * window_size, 10, 11, 1)
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, cost],
                               feed_dict={X: batch_x, Y: batch_y, keep_prob: 1 - dropout_prob, phase_train: True})
            cost_history = np.append(cost_history, c)
        if (epoch % 1 == 0):
            train_accuracy = np.zeros(shape=[0], dtype=float)
            test_accuracy = np.zeros(shape=[0], dtype=float)
            test_loss = np.zeros(shape=[0], dtype=float)
            train_loss = np.zeros(shape=[0], dtype=float)
            for i in range(train_accuracy_batch_num):
                offset = (i * accuracy_batch_size) % (train_y.shape[0] - accuracy_batch_size)
                train_batch_x = train_x[offset:(offset + accuracy_batch_size), :, :, :, :]
                train_batch_x = train_batch_x.reshape(len(train_batch_x) * window_size, 10, 11, 1)
                train_batch_y = train_y[offset:(offset + accuracy_batch_size), :]

                train_a, train_c = session.run([accuracy, cost],
                                               feed_dict={X: train_batch_x, Y: train_batch_y, keep_prob: 1.0,
                                                          phase_train: False})

                train_loss = np.append(train_loss, train_c)
                train_accuracy = np.append(train_accuracy, train_a)
            print("(" + time.asctime(time.localtime(time.time())) + ") Epoch: ", epoch + 1, " Training Cost: ",
                  np.mean(train_loss), "Training Accuracy: ", np.mean(train_accuracy))
            train_accuracy_save = np.append(train_accuracy_save, np.mean(train_accuracy))
            train_loss_save = np.append(train_loss_save, np.mean(train_loss))
            for j in range(test_accuracy_batch_num):
                offset = (j * accuracy_batch_size) % (test_y.shape[0] - accuracy_batch_size)
                test_batch_x = test_x[offset:(offset + accuracy_batch_size), :, :, :, :]
                test_batch_x = test_batch_x.reshape(len(test_batch_x) * window_size, 10, 11, 1)
                test_batch_y = test_y[offset:(offset + accuracy_batch_size), :]

                test_a, test_c = session.run([accuracy, cost],
                                             feed_dict={X: test_batch_x, Y: test_batch_y, keep_prob: 1.0,
                                                        phase_train: False})

                test_accuracy = np.append(test_accuracy, test_a)
                test_loss = np.append(test_loss, test_c)

            print("(" + time.asctime(time.localtime(time.time())) + ") Epoch: ", epoch + 1, " Test Cost: ",
                  np.mean(test_loss), "Test Accuracy: ", np.mean(test_accuracy), "\n")
            test_accuracy_save = np.append(test_accuracy_save, np.mean(test_accuracy))
            test_loss_save = np.append(test_loss_save, np.mean(test_loss))
    test_accuracy = np.zeros(shape=[0], dtype=float)
    test_loss = np.zeros(shape=[0], dtype=float)
    test_pred = np.zeros(shape=[0], dtype=float)
    test_true = np.zeros(shape=[0, 15], dtype=float)
    test_posi = np.zeros(shape=[0, 15], dtype=float)
    for k in range(test_accuracy_batch_num):
        offset = (k * accuracy_batch_size) % (test_y.shape[0] - accuracy_batch_size)
        test_batch_x = test_x[offset:(offset + accuracy_batch_size), :, :, :, :]
        test_batch_x = test_batch_x.reshape(len(test_batch_x) * window_size, 10, 11, 1)
        test_batch_y = test_y[offset:(offset + accuracy_batch_size), :]

        test_a, test_c, test_p, test_r = session.run([accuracy, cost, y_pred, y_posi],
                                                     feed_dict={X: test_batch_x, Y: test_batch_y, keep_prob: 1.0,
                                                                phase_train: False})
        test_t = test_batch_y  # one-hot 格式

        test_accuracy = np.append(test_accuracy, test_a)
        test_loss = np.append(test_loss, test_c)
        test_pred = np.append(test_pred, test_p)
        test_true = np.vstack([test_true, test_t])
        test_posi = np.vstack([test_posi, test_r])
#     test_true = tf.argmax(test_true, 1)
    test_pred_1_hot = np.asarray(pd.get_dummies(test_pred), dtype = np.int8)
    test_true_list = tf.argmax(test_true, 1).eval()
    # print(test_pred.shape)
    # print(test_pred)
    # print(test_true.shape)
    # print(test_true)
#     test_true = tf.argmax(test_true, 1)
#     test_true_np = test_true.eval()
#     # print(test_true_np)
#     test_true = test_true_np
#     test_true_list = test_true_np
    test_pred_1_hot = test_pred
    # print(test_pred.shape)
    # print(test_pred)
    
    y_score = test_posi
    n_classes = test_true.shape[1]
    test_size = len(y_score)
    total_comparisions = test_size * n_classes

    far, frr = prepare_graph_far_frr(test_true, y_score, total_comparisions - test_size, test_size)
    root = brentq(lambda x: x - interpolate.interp1d(far, frr)(x), min(far), 1.0)

    # recall
#     test_recall = recall_score(test_true, test_pred, average=None)
#     # precision
#     test_precision = precision_score(test_true, test_pred_1_hot, average=None)
#     # f1 score
#     test_f1 = f1_score(test_true, test_pred_1_hot, average=None)
    # auc
    # test_auc = roc_auc_score(test_true, test_pred_1_hot, average=None)
    # confusion matrix
    # 	confusion_matrix = confusion_matrix(test_true_list, test_pred)

    # plot_confusion_matrix(test_true, test_pred,[0,1,2,3,4,5,6,7,8])

#     print("********************recall:", test_recall)
#     print("*****************precision:", test_precision)
#     # print("******************test_auc:", test_auc)
#     print("******************f1_score:", test_f1)
    print("******************root:", root)
    # 	print("**********confusion_matrix:\n", confusion_matrix)

    print("(" + time.asctime(time.localtime(time.time())) + ") Final Test Cost: ", np.mean(test_loss),
          "Final Test Accuracy: ", np.mean(test_accuracy))
    # save result

    result = pd.DataFrame(
        {'epoch': range(1, epoch + 2), "train_accuracy": train_accuracy_save, "test_accuracy": test_accuracy_save,
         "train_loss": train_loss_save, "test_loss": test_loss_save})
    ins = pd.DataFrame({'conv_1': conv_1_shape, 'pool_1': pool_1_shape, 'conv_2': conv_2_shape, 'pool_2': pool_2_shape,
                        'conv_3': conv_3_shape, 'pool_3': pool_3_shape, 'conv_4': conv_4_shape, 'pool_3': pool_3_shape,
                        'fc': fc_size, 'accuracy': np.mean(test_accuracy), 'keep_prob': 1 - dropout_prob,
                        "calibration": calibration, 'sliding_window': window_size, "epoch": epoch + 1,
                        "norm": norm_type, "learning_rate": learning_rate, "regularization": regularization_method,
                        "train_sample": train_sample, "test_sample": test_sample}, index=[0])
    # summary = pd.DataFrame({'class':one_hot_labels, 'recall':test_recall, 'precision':test_precision, 'f1_score':test_f1, 'roc_auc':test_auc})

    writer = pd.ExcelWriter("./result/" + "output_file_p_15_32" + ".xlsx")
    ins.to_excel(writer, 'condition', index = False)
    result.to_excel(writer, 'result', index = False)
    # summary.to_excel(writer, 'summary', index=False)
    # fpr, tpr, auc
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    i = 0
    # for key in one_hot_labels:
    # 	fpr[key], tpr[key], _ = roc_curve(test_true[:, i], test_posi[:, i])
    # 	roc_auc[key] = auc(fpr[key], tpr[key])
    # 	roc = pd.DataFrame({"fpr":fpr[key], "tpr":tpr[key], "roc_auc":roc_auc[key]})
    # 	roc.to_excel(writer, key, index=False)
    # 	i += 1
    writer.save()
    model_dict = {}
    parameter_count = 0
    for variable in tf.trainable_variables():
        print(variable.name, "-->", variable.get_shape())
        count = 1
        for dim in variable.get_shape().as_list():
            count = count * dim
        parameter_count = parameter_count + count
        model_dict[variable.name] = session.run(variable)
    # 	sio.savemat(str(parameter_count)+".mat", model_dict)
    print("----------------------------------------------------------------")
    print("------------------total parameters", parameter_count, "-----------------------")
    print("----------------------------------------------------------------")

    print("**********(" + time.asctime(time.localtime(time.time())) + ") Train and Test NN End **********\n")
# 	pickle.dump(confusion_matrix, fp)
# # save model
# 	saver.save(session, './results_p_model_a_b/model_p_a_b.ckpt')


print("**********(" + time.asctime(time.localtime(time.time())) + ") Train and Test NN End **********\n")


W0831 16:26:49.665343 140599326492416 deprecation.py:506] From <ipython-input-3-8f0a34f26653>:140: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


train sample: 2997
test sample: 333
**********(Sat Aug 31 16:26:49 2019) Load and Split dataset End **********

**********(Sat Aug 31 16:26:49 2019) Define parameters and functions Begin: **********

**********(Sat Aug 31 16:26:49 2019) Define parameters and functions End **********

**********(Sat Aug 31 16:26:49 2019) Define NN structure Begin: **********

(?, 10, 11, 32)
(?, 10, 11, 64)
(?, 10, 11, 128)


W0831 16:26:50.070472 140599326492416 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0831 16:26:50.071725 140599326492416 deprecation.py:323] From <ipython-input-3-8f0a34f26653>:148: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0831 16:26:50.075576 140599326492416 deprecation.py:323] From <ipython-input-3-8f0a34f26653>:151: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructio

**********(Sat Aug 31 16:26:51 2019) Define NN structure End **********

**********(Sat Aug 31 16:26:51 2019) Train and Test NN Begin: **********

(Sat Aug 31 16:31:57 2019) Epoch:  1  Training Cost:  2.71699083453477 Training Accuracy:  0.06835017078603157
(Sat Aug 31 16:32:06 2019) Epoch:  1  Test Cost:  2.7331534949215976 Test Accuracy:  0.06363636627793312 

(Sat Aug 31 16:37:07 2019) Epoch:  2  Training Cost:  2.6226687768493036 Training Accuracy:  0.1393939434277891
(Sat Aug 31 16:37:17 2019) Epoch:  2  Test Cost:  2.6423285874453457 Test Accuracy:  0.112121216614138 

(Sat Aug 31 16:42:16 2019) Epoch:  3  Training Cost:  2.3221269906169235 Training Accuracy:  0.17272727752123215
(Sat Aug 31 16:42:25 2019) Epoch:  3  Test Cost:  2.3601999933069404 Test Accuracy:  0.14848485216498375 

(Sat Aug 31 16:47:25 2019) Epoch:  4  Training Cost:  2.1066719979950874 Training Accuracy:  0.22323232941856289
(Sat Aug 31 16:47:35 2019) Epoch:  4  Test Cost:  2.136072375557639 Test Accuracy:  0

(Sat Aug 31 19:41:15 2019) Epoch:  38  Training Cost:  0.10873626812385875 Training Accuracy:  0.9629629525271329
(Sat Aug 31 19:41:24 2019) Epoch:  38  Test Cost:  0.1364811279768632 Test Accuracy:  0.9606060548262163 

(Sat Aug 31 19:46:22 2019) Epoch:  39  Training Cost:  0.03318861822672353 Training Accuracy:  0.9868686789214008
(Sat Aug 31 19:46:31 2019) Epoch:  39  Test Cost:  0.059940969380973416 Test Accuracy:  0.9818181720646945 

(Sat Aug 31 19:51:28 2019) Epoch:  40  Training Cost:  0.5009382953396951 Training Accuracy:  0.8878787786069543
(Sat Aug 31 19:51:37 2019) Epoch:  40  Test Cost:  0.4968747676096179 Test Accuracy:  0.8818181698972528 

(Sat Aug 31 19:56:34 2019) Epoch:  41  Training Cost:  0.1952097610349419 Training Accuracy:  0.9407407310273912
(Sat Aug 31 19:56:43 2019) Epoch:  41  Test Cost:  0.13792361632328143 Test Accuracy:  0.9575757492672313 

(Sat Aug 31 20:01:40 2019) Epoch:  42  Training Cost:  0.18449995415332265 Training Accuracy:  0.9565656468121693
(

(Sat Aug 31 22:47:30 2019) Epoch:  75  Training Cost:  0.22744966235452488 Training Accuracy:  0.9632996543489322
(Sat Aug 31 22:47:39 2019) Epoch:  75  Test Cost:  0.24045419693522324 Test Accuracy:  0.9575757384300232 

(Sat Aug 31 22:52:35 2019) Epoch:  76  Training Cost:  0.6184659124530777 Training Accuracy:  0.8936026854948564
(Sat Aug 31 22:52:44 2019) Epoch:  76  Test Cost:  0.6991803203286096 Test Accuracy:  0.9060606035319242 

(Sat Aug 31 22:57:41 2019) Epoch:  77  Training Cost:  0.6683720805578761 Training Accuracy:  0.8249158209020441
(Sat Aug 31 22:57:50 2019) Epoch:  77  Test Cost:  0.6572681529955431 Test Accuracy:  0.81818180734461 

(Sat Aug 31 23:02:47 2019) Epoch:  78  Training Cost:  0.7517457162265224 Training Accuracy:  0.850841745583698
(Sat Aug 31 23:02:56 2019) Epoch:  78  Test Cost:  0.8109432242133401 Test Accuracy:  0.851515141400424 

(Sat Aug 31 23:07:53 2019) Epoch:  79  Training Cost:  0.06896409790204765 Training Accuracy:  0.9754208633393953
(Sat Aug